In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib

font = {"size": 15}
matplotlib.rc("font", **font)

## Version 1

Shows how CIs don't overlap in modelvshuman, but do overlap when we normalize by kappa_max. 

In [ ]:
# generate some demo data

rng = np.random.default_rng(seed=42)
df = pd.DataFrame(
    {
        "Metric": ["Standard EC"] * 200 + ["Corrected EC"] * 200,
        "Kappa": list(rng.normal(0.2, 0.1, size=100))
        + list(rng.normal(0.4, 0.1, size=100))
        + list(rng.normal(0.28, 0.21, size=100))
        + list(rng.normal(0.32, 0.21, size=100)),
        "Comparison Type": ["Human-Machine EC"] * 100
        + ["Human-Human EC"] * 100
        + ["Human-Machine EC"] * 100
        + ["Human-Human EC"] * 100,
    }
)

In [ ]:
# make demo figure one
with plt.xkcd():
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    sns.pointplot(
        data=df,
        x="Metric",
        y="Kappa",
        hue="Comparison Type",
        palette="mako",
        capsize=0.1,
        dodge=False,
        linestyles="dotted",
        ax=ax,
    )
    ax.set_ylim(0, 0.8)
    sns.despine()

    plt.tight_layout()
    plt.savefig(f"figures/demo_figure.pdf", bbox_inches="tight")
    plt.show()

## Version 2

Shows on the left how old results looked like with point estimates, and on the right how the new results will look like with CIs.

In [ ]:
# utility function


def axvline_with_caps(ax, x, ymin, ymax, cap_width=0.1, color="black", **line_kwargs):
    """
    Draws a vertical line with caps at the specified y-limits (in data coordinates).

    Parameters:
    - ax: matplotlib Axes object
    - x: x position of the vertical line (in data coordinates)
    - ymin: lower y-coordinate (in data coordinates)
    - ymax: upper y-coordinate (in data coordinates)
    - cap_width: width of the caps (in data coordinates)
    - **line_kwargs: additional keyword arguments passed to axvline and cap lines
    """
    # Get current axis limits
    y_axis_min, y_axis_max = ax.get_ylim()

    # Normalize ymin and ymax for axvline
    norm_ymin = (ymin - y_axis_min) / (y_axis_max - y_axis_min)
    norm_ymax = (ymax - y_axis_min) / (y_axis_max - y_axis_min)

    # Draw vertical line
    ax.axvline(x=x, ymin=norm_ymin, ymax=norm_ymax, color=color, **line_kwargs)

    # Draw horizontal caps at ends
    ax.plot(
        [x - cap_width / 2, x + cap_width / 2], [ymin, ymin], color=color, **line_kwargs
    )
    ax.plot(
        [x - cap_width / 2, x + cap_width / 2], [ymax, ymax], color=color, **line_kwargs
    )


def axhline_with_caps(ax, y, xmin, xmax, cap_height=0.05, color="black", **line_kwargs):
    """
    Draws a horizontal line with vertical caps at the specified x-limits (in data coordinates).

    Parameters:
    - ax: matplotlib Axes object
    - y: y position of the horizontal line (in data coordinates)
    - xmin: lower x-coordinate (in data coordinates)
    - xmax: upper x-coordinate (in data coordinates)
    - cap_height: height of the caps (in data coordinates)
    - **line_kwargs: additional keyword arguments passed to axhline and cap lines
    """
    # Get current axis limits
    x_axis_min, x_axis_max = ax.get_xlim()

    # Normalize xmin and xmax for axhline
    norm_xmin = (xmin - x_axis_min) / (x_axis_max - x_axis_min)
    norm_xmax = (xmax - x_axis_min) / (x_axis_max - x_axis_min)

    # Draw horizontal line
    ax.axhline(y=y, xmin=norm_xmin, xmax=norm_xmax, color=color, **line_kwargs)

    # Draw vertical caps at ends
    ax.plot(
        [xmin, xmin],
        [y - cap_height / 2, y + cap_height / 2],
        color=color,
        **line_kwargs,
    )
    ax.plot(
        [xmax, xmax],
        [y - cap_height / 2, y + cap_height / 2],
        color=color,
        **line_kwargs,
    )

In [ ]:
# data for old plot
np.random.seed(1)
df = pd.DataFrame(
    {
        "model": ["humans"] * 100 + ["A"] * 100 + ["B"] * 100 + ["C"] * 100,
        "EC": np.random.normal(0.42, 0.03, 100).tolist()
        + np.random.normal(0.23, 0.04, 100).tolist()
        + np.random.normal(0.2, 0.05, 100).tolist()
        + np.random.normal(0.18, 0.05, 100).tolist(),
    }
)


def plot():
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    an_color = "grey"

    # original plot on the left
    ax = axes[0]
    sns.pointplot(data=df, x="model", y="EC", linestyle="none", errorbar=None, ax=ax)
    sns.despine()
    ax.grid(axis="y")  # is ignored by xkcd style anyway
    ax.set_ylim(0, 0.55)
    ax.set_ylabel("EC [kappa]")
    ax.set_xlabel("Classifier")
    ax.set_title("Previous Work")

    # add annotations
    axvline_with_caps(ax, x=-0.15, ymin=0.23, ymax=0.42, color=an_color)
    ax.annotate("Human-Machine\nGap", (-0.1, 0.3), fontsize=12, color=an_color)

    axhline_with_caps(ax, y=0.11, xmin=1, xmax=3, color=an_color)
    ax.annotate("Model Ranking", (1.5, 0.05), fontsize=12, color=an_color)

    # new plot on the left
    ax = axes[1]
    sns.pointplot(
        data=df,
        x="model",
        y="EC",
        errorbar=("pi", 95),
        capsize=0.3,
        linestyle="none",
        ax=ax,
    )
    sns.despine()
    ax.grid(axis="y")
    ax.set_xlabel("Classifier")
    ax.set_title("Our Work")

    # add annotations
    axvline_with_caps(ax, x=-0.25, ymin=0.3, ymax=0.35, color=an_color)
    ax.annotate(
        " uncertain\n gap  size", (-0.14, 0.3), fontsize=12, color=an_color
    )  # extra whitespace is on purpose

    axhline_with_caps(ax, y=0.35, xmin=1, xmax=3, color=an_color)
    ax.annotate("insignificant  differences", (1.0, 0.4), fontsize=12, color=an_color)

    plt.tight_layout()
    plt.savefig(f"figures/demo_figure_v2.pdf", bbox_inches="tight")
    plt.show()


with plt.xkcd():
    plot()